In [6]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [7]:
# Создаём сэмпл
n_samples = 500

Я сделаю жестокий пример: вероятность серфера умереть в зависимости от высоты волны и процента каменистости дна

In [31]:
#день года
year_day = np.random.choice(365, n_samples) + 1

#высота волны в см
wave_height = np.random.choice(1200, n_samples)

#общая площадь берега
square = np.random.choice(2000, n_samples) + 100

#площадь камней
rocks = np.array([])
for x in square:
    rocks = np.append(rocks, np.random.randint(x - 100) + 100)  

death_rate = 0.5 / 1200 * wave_height + 0.5 * rocks / square  

data = pd.DataFrame({'year_day': year_day, 'wave_height': wave_height, 'square': square, 'rocks': rocks, 'death_rate': death_rate})
data.head(5)

,year_day,wave_height,square,rocks,death_rate
0,177,914,1393,861.0,0.689879
1,203,443,1823,719.0,0.381786
2,104,106,2032,771.0,0.233881
3,310,1117,1217,418.0,0.637150
4,42,211,1658,1561.0,0.558665


Первым шагом пробуем линейную регрессию на все параметры

In [32]:
from sklearn.metrics import mean_absolute_error

X = data[['year_day', 'wave_height', 'square', 'rocks']]
y = data['death_rate']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['year_day', 'wave_height', 'square', 'rocks']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-3.72548336e-06  4.26410378e-04 -2.26326152e-04  3.47751629e-04]
Bias: 0.3249596615406196
Error: 0.045003704182343174


In [33]:
data.death_rate.mean()

0.5493127053771807

Ошибка около 10%, что в общем то не так уж много, но...
Удаляем из обучения признак "день года", маловероятно что он как то влияет, если только не верить в астрологию)

In [34]:
X = data[['wave_height', 'square', 'rocks']]

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['wave_height', 'square', 'rocks']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.00042637 -0.00022636  0.00034768]
Bias: 0.32436580697983636
Error: 0.04499451141880026


Ошибка не сильно меньше стало, но оно не удивительно, вес года модель изначально определила очень маленьким

In [36]:
# Создаем новый признак из square и rocks, все-таки по отдельности они практически не имеют смысла. 
#Важна не общая площадь камней, а насколько дно каменистое
data['danger'] = data['rocks'] / data['square']
data.head(5)

,year_day,wave_height,square,rocks,death_rate,danger
0,177,914,1393,861.0,0.689879,0.618090
1,203,443,1823,719.0,0.381786,0.394405
2,104,106,2032,771.0,0.233881,0.379429
3,310,1117,1217,418.0,0.637150,0.343468
4,42,211,1658,1561.0,0.558665,0.941496


In [37]:
X = data[['wave_height', 'danger']]

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['wave_height', 'danger']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [4.16666667e-04 5.00000000e-01]
Bias: 0.0
Error: 1.3043732760564807e-16


Эх, так бы в жизни курс валют предсказывать :)